In [6]:
import random
import time
import decimal
from decimal import Decimal, ROUND_HALF_UP
from z3 import Real, Solver, If, sat
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from fractions import Fraction

# 定義一個模擬人類逐字輸入的函式
def human_type(element, text, delay=0.1):
    """逐字輸入 text 到 element，每個字間隔 delay 秒，最後送出 TAB"""
    for char in text:
        element.send_keys(char)
        time.sleep(delay)
    element.send_keys(Keys.TAB)


def compute_corporate_income_tax_z3(
    OperatingRevenueTotal_input=0, SalesReturn_input=0, SalesAllowance_input=0,
    OperatingCost_input=0, OperatingExpensesLosses_input=0,
    NonOperatingRevenueTotal_input=0, NonOperatingLossExpenses_input=0,
    Prev10LossDeduction_input=0, TaxIncentiveExempt_input=0,
    ExemptSecuritiesIncome_input=0, ExemptLandIncome_input=0,
    Article4_4HouseLandGain_input=0,
    is_full_year=True,    # 營業期間是否滿一年
    m_partial=12          # 營業期間未滿一年時，營業月份數
):
    s = Solver()

    # --- 1. 定義 12 個變數 ---
    OperatingRevenueTotal    = Real('OperatingRevenueTotal')
    SalesReturn              = Real('SalesReturn')
    SalesAllowance           = Real('SalesAllowance')
    OperatingCost            = Real('OperatingCost')
    OperatingExpensesLosses  = Real('OperatingExpensesLosses')
    NonOperatingRevenueTotal = Real('NonOperatingRevenueTotal')
    NonOperatingLossExpenses = Real('NonOperatingLossExpenses')
    Prev10LossDeduction      = Real('Prev10LossDeduction')
    TaxIncentiveExempt       = Real('TaxIncentiveExempt')
    ExemptSecuritiesIncome   = Real('ExemptSecuritiesIncome')
    ExemptLandIncome         = Real('ExemptLandIncome')
    Article4_4HouseLandGain  = Real('Article4_4HouseLandGain')

    s.add(OperatingRevenueTotal    == OperatingRevenueTotal_input)
    s.add(SalesReturn              == SalesReturn_input)
    s.add(SalesAllowance           == SalesAllowance_input)
    s.add(OperatingCost            == OperatingCost_input)
    s.add(OperatingExpensesLosses  == OperatingExpensesLosses_input)
    s.add(NonOperatingRevenueTotal == NonOperatingRevenueTotal_input)
    s.add(NonOperatingLossExpenses == NonOperatingLossExpenses_input)
    s.add(Prev10LossDeduction      == Prev10LossDeduction_input)
    s.add(TaxIncentiveExempt       == TaxIncentiveExempt_input)
    s.add(ExemptSecuritiesIncome   == ExemptSecuritiesIncome_input)
    s.add(ExemptLandIncome         == ExemptLandIncome_input)
    s.add(Article4_4HouseLandGain  == Article4_4HouseLandGain_input)

    # --- 2. 中間計算變數 ---
    OperatingRevenueNet   = Real('OperatingRevenueNet')
    OperatingGrossProfit  = Real('OperatingGrossProfit')
    OperatingNetProfit    = Real('OperatingNetProfit')
    YearlyIncome          = Real('YearlyIncome')
    T                     = Real('T')

    s.add(OperatingRevenueNet  == OperatingRevenueTotal - SalesReturn - SalesAllowance)
    s.add(OperatingGrossProfit == OperatingRevenueNet - OperatingCost)
    s.add(OperatingNetProfit   == OperatingGrossProfit - OperatingExpensesLosses)
    s.add(YearlyIncome         == OperatingNetProfit + NonOperatingRevenueTotal - NonOperatingLossExpenses)

    # --- 3. 分支計算應納稅額 T ---
    P = Real('P')
    s.add(P == YearlyIncome 
        - Prev10LossDeduction 
        - TaxIncentiveExempt 
        - ExemptSecuritiesIncome 
        - ExemptLandIncome 
        - Article4_4HouseLandGain)
    
    if is_full_year:
        T_expr = If(P <= 120000,
                    0,
                    If(P <= 200000,
                       (P - 120000) * 0.5,
                       P * 0.20))
    else:
        P_adj = Real('P_adj')
        s.add(P_adj == P * 12 / m_partial)
        T_expr = If(P_adj <= 120000,
                    0,
                    If(P_adj <= 200000,
                       (P_adj - 120000) * 0.5 * (m_partial / 12),
                       P_adj * 0.20 * (m_partial / 12)))
    piecewise_T = Real('piecewise_T')
    s.add(piecewise_T == T_expr)
    s.add(T == piecewise_T)

    if s.check() == sat:
        model = s.model()
        # 如果模型返回的 T 是一個有理數，則可以用 numerator_as_long 和 denominator_as_long
        r = model[T]
        try:
            num = r.numerator_as_long()
            den = r.denominator_as_long()
            frac = Fraction(num, den)
        except Exception:
            # 如果沒有這個方法，則退回 as_decimal 轉換 (但通常有理數可以這麼取)
            r_str = r.as_decimal(50)
            if r_str.endswith('?'):
                r_str = r_str[:-1]
            frac = Fraction(r_str)
        # 這裡我們依照需求直接取整（即捨去小數部分，不進行四捨五入）
        return int(frac)
    else:
        raise ValueError("Z3 無法求解或約束衝突。")


# ========== Selenium 測試自動化 ==========
def run_corporate_tax_test():
    driver = webdriver.Chrome()
    driver.delete_all_cookies()
    driver.get("https://www.etax.nat.gov.tw/etwmain/etw158w/35")
    
    total_tests = 0
    match_count = 0

    for i in range(100):
        print(f"===== 迴圈第 {i+1} 次 =====")
        # 隨機產生參數（皆為整數）
        while True:
            OperatingRevenueTotal_input    = random.randint(1_000_000, 10_000_000)
            SalesReturn_input              = random.randint(0, 500_000)
            SalesAllowance_input           = random.randint(0, 200_000)
            OperatingCost_input            = random.randint(1_000_000, 8_000_000)
            OperatingExpensesLosses_input  = random.randint(0, 3_000_000)
            NonOperatingRevenueTotal_input = random.randint(0, 1_000_000)
            NonOperatingLossExpenses_input = random.randint(0, 500_000)
            Prev10LossDeduction_input      = random.randint(0, 1_000_000)
            TaxIncentiveExempt_input       = random.randint(0, 100_000)
            ExemptSecuritiesIncome_input   = random.randint(0, 100_000)
            ExemptLandIncome_input         = random.randint(0, 100_000)
            Article4_4HouseLandGain_input  = random.randint(0, 50_000)

            # # 隨機決定是否滿一年
            # is_full_year = random.choice([True, False])
            # if is_full_year:
            #     select_val = "Y"
            #     m_partial = 12
            # else:
            #     select_val = "N"
            #     m_partial = random.randint(1, 11)

            # 為了測試特定情況使用這一段
            # 固定設定未滿一年，並從指定列表中選取 m_partial
            is_full_year = False
            select_val = "N"
            m_partial = random.choice([1, 2, 4, 5, 7, 8, 10, 11])


            # 計算 YearlyIncome 和 P
            OperatingRevenueNet = OperatingRevenueTotal_input - SalesReturn_input - SalesAllowance_input
            OperatingGrossProfit = OperatingRevenueNet - OperatingCost_input
            OperatingNetProfit = OperatingGrossProfit - OperatingExpensesLosses_input
            YearlyIncome = OperatingNetProfit + NonOperatingRevenueTotal_input - NonOperatingLossExpenses_input
            P = YearlyIncome - Prev10LossDeduction_input - TaxIncentiveExempt_input - ExemptSecuritiesIncome_input - ExemptLandIncome_input - Article4_4HouseLandGain_input

            if YearlyIncome > 0 and P > 0:
                break  # 符合條件，跳出迴圈

        # 取得本地計算結果
        try:
            local_tax = compute_corporate_income_tax_z3(
                OperatingRevenueTotal_input, SalesReturn_input, SalesAllowance_input,
                OperatingCost_input, OperatingExpensesLosses_input,
                NonOperatingRevenueTotal_input, NonOperatingLossExpenses_input,
                Prev10LossDeduction_input, TaxIncentiveExempt_input,
                ExemptSecuritiesIncome_input, ExemptLandIncome_input,
                Article4_4HouseLandGain_input,
                is_full_year=is_full_year,
                m_partial=m_partial
            )
            local_tax_int = local_tax  # 此處 local_tax 已經直接取 int（捨去小數部分）
        except Exception as e:
            print(f"Local calc error: {e}")
            local_tax_int = None

        # 填寫表單欄位 (模擬人類逐字輸入並以 TAB 移到下一欄)
        try:
            driver.delete_all_cookies()
            driver.get("https://www.etax.nat.gov.tw/etwmain/etw158w/35")
            # 1. 營業收入
            elem = driver.find_element(By.ID, "operatingIncome")
            elem.clear()
            human_type(elem, str(OperatingRevenueTotal_input))
            
            # 2. 銷售退回
            elem = driver.find_element(By.ID, "salesReturns")
            elem.clear()
            human_type(elem, str(SalesReturn_input))
            
            # 3. 銷售折讓
            elem = driver.find_element(By.ID, "salesDiscount")
            elem.clear()
            human_type(elem, str(SalesAllowance_input))
            
            # 4. 營業成本
            elem = driver.find_element(By.ID, "operatingCosts")
            elem.clear()
            human_type(elem, str(OperatingCost_input))
            
            # 5. 營業費用損失
            elem = driver.find_element(By.ID, "operatingTotal")
            elem.clear()
            human_type(elem, str(OperatingExpensesLosses_input))
            
            # 6. 非營業收入
            elem = driver.find_element(By.ID, "notIncome")
            elem.clear()
            human_type(elem, str(NonOperatingRevenueTotal_input))
            
            # 7. 非營業費用損失
            elem = driver.find_element(By.ID, "notOperatingTotal")
            elem.clear()
            human_type(elem, str(NonOperatingLossExpenses_input))
            
            # 8. 過去10年虧損扣抵
            elem = driver.find_element(By.ID, "declare1")
            elem.clear()
            human_type(elem, str(Prev10LossDeduction_input))
            
            # 9. 稅收優惠免稅額
            elem = driver.find_element(By.ID, "declare2")
            elem.clear()
            human_type(elem, str(TaxIncentiveExempt_input))
            
            # 10. 證券免稅收入
            elem = driver.find_element(By.ID, "declare3")
            elem.clear()
            human_type(elem, str(ExemptSecuritiesIncome_input))
            
            # 11. 土地免稅收入
            elem = driver.find_element(By.ID, "declare4")
            elem.clear()
            human_type(elem, str(ExemptLandIncome_input))
            
            # 12. 第五項免稅收入
            elem = driver.find_element(By.ID, "declare5")
            elem.clear()
            human_type(elem, str(Article4_4HouseLandGain_input))
            
            # 設定下拉選單：營業期間是否滿一年
            select_elem = driver.find_element(By.ID, "selectResult")
            Select(select_elem).select_by_value(select_val)
            
            # 若未滿一年，填入月份數 (模擬人類輸入)
            if not is_full_year:
                elem = driver.find_element(By.ID, "month")
                elem.clear()
                human_type(elem, str(m_partial))
        except Exception as e:
            print(f"Form fill error: {e}")
        
        # 等待所有欄位完成填入後，自動計算完成（此處等待 2 秒，可依需求調整）
        time.sleep(2)
        
        # 讀取結果：統一取得最後一個 <span class="text-danger col-auto"> 的值
        try:
            elements = driver.find_elements(By.CSS_SELECTOR, "span.text-danger.col-auto")
            if not elements:
                raise Exception("找不到任何結果元素")
            res_str = elements[-1].text
            res_str = res_str.replace(",", "").strip()
            online_tax_int = int(Decimal(str(float(res_str))))
        except Exception as e:
            print(f"結果取得錯誤: {e}")
            online_tax_int = None
        
        total_tests += 1
        if local_tax_int is not None and online_tax_int is not None and local_tax_int == online_tax_int:
            match_count += 1
            print(f"Test {i+1}: Local Tax = {local_tax_int}, Online Tax = {online_tax_int}")
        else:
            print(f"Test {i+1}: Local Tax = {local_tax_int}, Online Tax = {online_tax_int}")
            print(f"  --> Mismatch! Parameters: {{'OperatingRevenueTotal': {OperatingRevenueTotal_input}, 'SalesReturn': {SalesReturn_input}, 'SalesAllowance': {SalesAllowance_input}, 'OperatingCost': {OperatingCost_input}, 'OperatingExpensesLosses': {OperatingExpensesLosses_input}, 'NonOperatingRevenueTotal': {NonOperatingRevenueTotal_input}, 'NonOperatingLossExpenses': {NonOperatingLossExpenses_input}, 'Prev10LossDeduction': {Prev10LossDeduction_input}, 'TaxIncentiveExempt': {TaxIncentiveExempt_input}, 'ExemptSecuritiesIncome': {ExemptSecuritiesIncome_input}, 'ExemptLandIncome': {ExemptLandIncome_input}, 'Article4_4HouseLandGain': {Article4_4HouseLandGain_input}, 'is_full_year': {is_full_year}, 'm_partial': {m_partial} }}")
        print("\n")
    
    match_percentage = (match_count / total_tests) * 100 if total_tests > 0 else 0
    print(f"總共匹配次數: {match_count} / {total_tests}  ({match_percentage:.2f}%)")
    driver.quit()

if __name__ == "__main__":
    run_corporate_tax_test()


===== 迴圈第 1 次 =====
Test 1: Local Tax = 183371, Online Tax = 0
  --> Mismatch! Parameters: {'OperatingRevenueTotal': 7847571, 'SalesReturn': 27016, 'SalesAllowance': 184753, 'OperatingCost': 5936754, 'OperatingExpensesLosses': 812806, 'NonOperatingRevenueTotal': 946058, 'NonOperatingLossExpenses': 104966, 'Prev10LossDeduction': 717229, 'TaxIncentiveExempt': 44380, 'ExemptSecuritiesIncome': 6753, 'ExemptLandIncome': 21822, 'Article4_4HouseLandGain': 20293, 'is_full_year': False, 'm_partial': 1 }


===== 迴圈第 2 次 =====
Test 2: Local Tax = 480879, Online Tax = 480879


===== 迴圈第 3 次 =====
Test 3: Local Tax = 36383, Online Tax = 36383


===== 迴圈第 4 次 =====
Test 4: Local Tax = 261246, Online Tax = 261246


===== 迴圈第 5 次 =====
Test 5: Local Tax = 164600, Online Tax = 164600


===== 迴圈第 6 次 =====
Test 6: Local Tax = 155503, Online Tax = 155503


===== 迴圈第 7 次 =====
Test 7: Local Tax = 863980, Online Tax = 863980


===== 迴圈第 8 次 =====
Test 8: Local Tax = 24928, Online Tax = 24928


===== 迴圈第 9 